In [1]:
import torch
import numpy as np

a = torch.FloatTensor(3,2)
a

tensor([[ 0.0000e+00, -0.0000e+00],
        [ 4.7668e-04, -4.6577e-10],
        [ 4.2981e+21,  6.3828e+28]])

In [2]:
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [3]:
torch.FloatTensor([[1,2,3],[3,2,1]])

tensor([[1., 2., 3.],
        [3., 2., 1.]])

In [4]:
n = np.zeros(shape=(3,2))
n

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [5]:
b = torch.tensor(n)
b

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]], dtype=torch.float64)

In [6]:
n = np.zeros(shape=(3,2), dtype=np.float32)
torch.tensor(n)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [7]:
n = np.zeros(shape=(3,2))
torch.tensor(n, dtype=torch.float32)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [8]:
a = torch.tensor([1,2,3])
a

tensor([1, 2, 3])

In [9]:
s = a.sum()
s

tensor(6)

In [10]:
s.item()

6

In [11]:
torch.tensor(1)

tensor(1)

In [12]:
a = torch.FloatTensor([2,3])
a

tensor([2., 3.])

In [13]:
ca = a.cuda()
ca

RuntimeError: Cannot initialize CUDA without ATen_cuda library. PyTorch splits its backend into two shared libraries: a CPU library and a CUDA library; this error has occurred because you are trying to use some CUDA functionality, but the CUDA library has not been loaded by the dynamic linker for some reason.  The CUDA library MUST be loaded, EVEN IF you don't directly use any symbols from the CUDA library! One common culprit is a lack of -Wl,--no-as-needed in your link arguments; many dynamic linkers will delete dynamic library dependencies if you don't depend on any of their symbols.  You can check if this has occurred by using ldd on your binary to see if there is a dependency on *_cuda.so library.

In [14]:
torch.cuda.is_available()

False

In [15]:
v1 = torch.tensor([1.0, 1.0], requires_grad=True)
v2 = torch.tensor([2.0, 2.0])

v_sum = v1 + v2
v_res = (v_sum*2).sum()
v_res

tensor(12., grad_fn=<SumBackward0>)

In [16]:
v1.is_leaf, v2.is_leaf

(True, True)

In [17]:
v_sum.is_leaf, v_res.is_leaf

(False, False)

In [18]:
v1.requires_grad

True

In [19]:
v2.requires_grad

False

In [20]:
v_sum.requires_grad

True

In [21]:
v_res.requires_grad

True

In [22]:
v_res.backward()
v1.grad

tensor([2., 2.])

In [23]:
v2.grad

In [24]:
v_sum.grad

In [25]:
import torch.nn as nn

l = nn.Linear(2, 5)
v = torch.FloatTensor([1,2])
l(v)

tensor([ 2.1411,  1.5217, -0.7269,  0.3728, -0.2233], grad_fn=<ThAddBackward>)

In [26]:
s = nn.Sequential(nn.Linear(2,5), 
                   nn.ReLU(),
                   nn.Linear(5,20), 
                   nn.ReLU(), 
                   nn.Linear(20,10), 
                   nn.Dropout(p=0.3), 
                   nn.Softmax(dim=1))
s

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): Dropout(p=0.3)
  (6): Softmax()
)

In [27]:
s(torch.FloatTensor([[1,2]]))

tensor([[0.1489, 0.0986, 0.0518, 0.1238, 0.0681, 0.0692, 0.0951, 0.0951, 0.1537,
         0.0957]], grad_fn=<SoftmaxBackward>)

In [28]:
class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob=0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 5),
            nn.ReLU(),
            nn.Linear(5,20),
            nn.ReLU(),
            nn.Linear(20, num_classes),
            nn.Dropout(p=dropout_prob),
            nn.Softmax()
        )
        
    def forward(self, x):
        return self.pipe(x)
    
net = OurModule(num_inputs = 2, num_classes = 3)
v = torch.FloatTensor([[2,3]])
out = net(v)
print(net)
print(out)

OurModule(
  (pipe): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
    (5): Dropout(p=0.3)
    (6): Softmax()
  )
)
tensor([[0.3449, 0.2950, 0.3601]], grad_fn=<SoftmaxBackward>)


/Users/karel/opt/anaconda3/envs/DRL-Book/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [29]:
import math
from tensorboardX import SummaryWriter

writer = SummaryWriter()

funcs = {"sin":math.sin, "cos":math.cos, "tan": math.tan}

for angle in range(-360,360):
    angle_rad = angle * math.pi / 180
    for name, fun in funcs.items():
        val = fun(angle_rad)
        writer.add_scalar(name, val, angle)
writer.close()

## Practical Example

In [31]:
import gym

class InputWrapper(gym.ObservationWrapper):
    def __init__(self, *args):
        super(InputWrapper, self).__init__(*args)
        assert isinstance(self.observation_space, gym.spaces.Box)
        old_space = self.observation_space
        self.observation_space = gym.spaces.Box(self.observation(old_space.low),
                                               self.observation(old_space.high),
                                               dtype=np.float32)
    
    def observation(self, observation):
         #resize image
        new_obs = cv2.resize(observation, (IMAGE_SIZE, IMAGE_SIZE))
        
        #transform (210,160,3) -> (3,210,160)
        new_obs = np.moveaxis(new_obs, 2, 0)
        
        return new_obs.astype(np.float32) / 255.0

In [32]:
def iterate_batches(envs, batch_size=BATCH_SIZE):
    batch = [e.reset() for e in envs]
    env_gen = iter(lambda: random.choice(envs), None)
    
    while True:
        e = next(env_gen)
        obs, reward, is_done, _ = e.step(e.action_space.sample())
        if np.mean(obs) > 0.01:
            batch.append(obs)
            
        if len(batch) == batch_size:
            yield torch.FloatTensor(batch)
            batch.clear()
        if is_done:
            e.reset()

NameError: name 'BATCH_SIZE' is not defined

In [33]:
parser = argparse.ArgumentParser()
parser.add_argument("--cuda", default=False,action='store_true')
args = parser.parse_args()
device = torch.device("cuda" if args.cuda else "cpu")

env_names = ('Breakout-v0', 'AirRaid-v0', 'Pong-v0')
envs = [InputWrapper(gym.make(name)) for name in env_names]
input_shape = envs[0].observation_space.shape

NameError: name 'argparse' is not defined

In [34]:
Writer = SummaryWriter()
net_discr = Discriminator(input_shape=input_shape).to(device)
net_gener = Generator(output_shape=input_shape).to(device)

objective = nn.BCELoss()
gen_optimizer = optim.Adam(params=net_gener.parameters(), lr=LEARNING_RATE)
dis_optimizer = optim.Adam(params=net_discr.paramaters(), lr=LEARNING_RATE)

NameError: name 'Discriminator' is not defined

In [35]:
gen_losses = []
dis_losses = []
inter_no = 0

true_labels_v = torch.ones(BATCH_SIZE, dtype=torch.float32, device=device)
fake_labels_v = torch.zeros(BATCH_SIZE, dtype=torch.float32, device=device)

NameError: name 'BATCH_SIZE' is not defined

In [36]:
for batch_v in iterate_batches(envs):
    #generate extra fake samples, input is 4D: batch, filters, x, y
    
    gen_input_v = torch.FloatTensor(BATCH_SIZE, LATENT_VECTOR_SIZE, 1, 1).normal_(0, 1).to(device)
    batch_v = batch_v.to(device)
    gen_output_v = net_gener(gen_input_v)

NameError: name 'iterate_batches' is not defined

In [37]:
dis_optimizer.zero_grad()
dis_output_true_v = net_discr(batch_v)
dis_output_fake_v = net_discr(gen_output_v.detach())
dis_loss = objective(dis_output_true_v, true_labels_v) + objective(dis_output_fake_v, fake_labels_v)
dis_loss.backward()
dis_optimizer.step()
dis_losses.append(dis_loss.item())

NameError: name 'dis_optimizer' is not defined

In [38]:
gen_optimizer.zero_grad()
dis_output_v = net_discr(gen_output_v)
gen_loss_v = objective(dis_output_v, true_labels_v)
gen_loss_v.backward()
gen_optimizer.step()
gen_losses.append(gen_loss_v.item())

NameError: name 'gen_optimizer' is not defined